In [ ]:
import math
import numpy as np
from scipy.linalg import *
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame,Series

In [ ]:
rho = 2700 #kg/m³
cp = 900 #J/kg.°C
k = 230 #W/m°C
ri = 0.050 #m
re = 0.056 #m
T_infinito = 80 #°C
h = 20000 #W/m².°C
Ti = 80 #°C
H = 1 #m²

pi = math.pi

In [ ]:
def T_sup(t):
    T_sup = 150 + 50 * math.cos(50 * pi * t)
    return T_sup

In [ ]:
mesh = 5
deltar = (re-ri)/(mesh-1)

time = 650
deltat = 0.002

In [ ]:
def TDMA(A,B):   
# o vetor de coeficientes P poderia ser definida
#no procedimento principal, mas para fazer um método 
#genérico de resolução TDMA ela é definida aqui
    P = np.zeros(mesh)
    Q = np.zeros(mesh)
    #T = np.zeros((mesh,1))  
    T = np.zeros(mesh)
    Matrix_P = np.zeros((mesh,mesh))
    Matrix_Q = np.zeros((mesh,1))
    
    #print(B)
    
    Q[0] = B[0]/A[0,0]  
    P[0] = A[0,1]/A[0,0]
    

    for i in range(1,mesh):        
        Q[i] = (B[i] + A[i,i-1]*Q[i-1])/(A[i,i] - A[i,i-1]*P[i-1])
        if (i<mesh-1):
            P[i] = A[i,i+1]/(A[i,i] - A[i,i-1]*P[i-1])
        
        
    '''
    for i in range(mesh):
        Matrix_P[i,i] = 1            
        Matrix_Q[i,0] = Q[i]
        
        if (i< mesh-1):
            Matrix_P[i,i+1] = -P[i]
    T = solve(Matrix_P,Matrix_Q)  
    '''
    T[mesh-1] = Q[mesh-1]
    for i in range(mesh-2,-1,-1):
        T[i] = P[i] *T[i+1] + Q[i]
        
       
    
    
    #print(Matrix_Q)
    #return T.transpose()
    return T


In [ ]:
#método numérico
T = np.zeros((time,mesh))
q = np.zeros((time,mesh))
q_A = np.zeros((time,mesh))
#M = rho * H * deltar
x_plot = np.zeros(mesh)
t_plot = np.zeros(time)

Matrix_coef_a = np.zeros((mesh,mesh))
Matrix_coef_b = np.zeros((mesh,1))

x_plot[0] = ri
t_plot[0] = 0
for volume in range(mesh):
    T[0,volume] = Ti 
    if volume>0:
        x_plot[volume] = x_plot[volume-1] +deltar

T[0,0] = T_sup(0)
'''
for t in range (time):
    if t>0:
        t_plot[t] = t_plot[t-1] + deltat
'''
#Definição da matriz de coeficientes        
for r in range(mesh): 
    if (r == 0):
        ap = 1                
        Matrix_coef_a[0,0] = ap
                    
    if (r > 0) and (r < mesh-1):
        ae = k*H*2*pi*(x_plot[r]+deltar/2)/deltar #malha igualmente espaçada
        aw = k*H*2*pi*(x_plot[r]-deltar/2)/deltar
        ap0 = rho*cp*pi*H*((x_plot[r]+deltar/2)**2 - (x_plot[r]-deltar/2)**2)/deltat
        ap = ap0 + ae + aw
            
        Matrix_coef_a[r,r-1] = aw #coloquei como positivo para utilizar no TDMA
        Matrix_coef_a[r,r+1] =ae #não precisaria definir matrizes de coef A e B,
        Matrix_coef_a[r,r] = ap #porém fiz essa opção para obter um método
        #Matrix_coef_b[r,0] = b #genérico de TDMA
            
            
    if (r == mesh-1):
        aw = k*H*2*pi*(x_plot[r]-deltar/2)/deltar
        ap0 = rho*cp*pi*H*(x_plot[r]**2 - (x_plot[r]-deltar/2)**2)/deltat
        ap = ap0 + aw + 2*pi*H*h*x_plot[r]
        #b = 2*pi*re*H*h*T_infinito
            
        Matrix_coef_a[r,r-1] = aw
        Matrix_coef_a[r,r] = ap
t = 1 
eps = 1
while (eps > 1e-3):            
#for t in range(1,time):
    for r in range(mesh):
        if (r==0):            
            b = T_sup(t*deltat)            
            
            Matrix_coef_b[0,0] = b 
            
        if (r > 0) and (r < mesh-1):
            ap0 = rho*cp*pi*H*((x_plot[r]+deltar/2)**2 - (x_plot[r]-deltar/2)**2)/deltat
            
            Matrix_coef_b[r,0] = ap0*T[t-1,r]
            
        if (r == mesh-1):
            ap0 = rho*cp*pi*H*(x_plot[r]**2 - (x_plot[r]-deltar/2)**2)/deltat
            b = 2*pi*re*H*h*T_infinito
            Matrix_coef_b[r,0] = b + ap0*T[t-1,r]
        
        
    T[t,:] = TDMA(Matrix_coef_a,Matrix_coef_b)
       
    #if (t_plot[t] > 0.04):
    eps = abs(T[t,mesh-1] - T[(t-(0.04/deltat)),mesh-1])
    t_plot[t] = t_plot[t-1] + deltat
    t += 1
    
for t in range(time):
    for r in range(mesh-1):
        q[t,r] = k*2*pi*(x_plot[r]-deltar/2)*H*(T[t,r]-T[t,r+1])/deltar
        q_A[t,r] = k*(T[t,r]-T[t,r+1])/deltar
    q[t,mesh-1] = h*2*pi*re*H*(T[t,mesh-1] - T_infinito)
    q_A[t,mesh-1] = h*(T[t,mesh-1] - T_infinito)        
                

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(8,6))
axis = fig.add_subplot(111)  

#for x in range(mesh):    
    #axis.plot(t_plot,T[:,x],label='x = '+str(x_plot[x]) + ' m')
    #axis.plot(t_plot,q[:,x],label='x = '+str(x_plot[x]) + ' m')
    

T_max = np.zeros(mesh)
T_min = np.ones(mesh)*300
for x in range(mesh):
    for i in range(580,600):
        if (T[i,x] > T_max[x]):
            T_max[x] = T[i,x]
        if (T[i,x] < T_min[x]):
            T_min[x] = T[i,x]       
        
    
axis.plot(x_plot,T_max,label = 'Temperatura máxima',marker ='o')    
axis.plot(x_plot,T_min,label = 'Temperatura mínima',marker ='s')
#axis.set_ylabel('q [W]')    
axis.set_ylabel('T[°C]')
axis.set_xlabel('x [m]')
axis.set_title('')

axis.ticklabel_format(style = 'plain')

axis.legend(loc="best")
#axis.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
#axis.set_ylim(124.5,125.5)
#axis.set_xlim(1.15,1.2)
